In [1]:
!pip install datasets
!pip install trl
!pip install peft
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.4 MB/s eta 0:00:00


In [2]:
import torch
import gc

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig, TaskType, get_peft_model

In [3]:
torch.cuda.is_available()

True

In [4]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-small")

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("csv", data_files="csv_moduls.csv", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset.to_json('df.jsonl')


Creating json from Arrow format:   0%|          | 0/815 [00:00<?, ?ba/s]

95012778

In [ ]:
dataset['prompt']

['private static @NotNull Icon load(@NotNull String path, int cacheKey, int flags) { return IconManager.',
 'private static @NotNull Icon load(@NotNull String path, int cacheKey, int flags) { return IconManager.getInstance().',
 'private static @NotNull Icon load(@NotNull String path, int cacheKey, int flags) { return IconManager.getInstance().loadRasterizedIcon(path, JupyterCoreIcons.class.',
 'public void setUp() throws IOException { System.',
 'public void setUp() throws IOException { System.setProperty("temp.dir", temporaryFolder.',
 'public void setUp() throws IOException { System.setProperty("temp.dir", temporaryFolder.getRoot().',
 'System.setProperty("temp.dir", temporaryFolder.getRoot().getCanonicalPath()); buildRule.',
 'public void smoke() { buildRule.',
 'buildRule.executeTarget("test.smoke"); String log = buildRule.',
 'String log = buildRule.getLog(); Assert.',
 'String log = buildRule.getLog(); Assert.assertTrue(log, log.',
 'Assert.assertTrue(log, log.contains("is missi

In [27]:
dataset['module']

['getInstance(',
 'loadRasterizedIcon(',
 'getClassLoader(',
 'setProperty(',
 'getRoot(',
 'getCanonicalPath(',
 'configureProject(',
 'executeTarget(',
 'getLog(',
 'assertTrue(',
 'contains(',
 'assertTrue(',
 'contains(',
 'assertTrue(',
 'contains(',
 'get(',
 'getProperty(',
 'resolve(',
 'createPathElement(',
 'setPath(',
 'toString(',
 'assertEquals(',
 'buildManifest(',
 'append(',
 'setRefid(',
 'append(',
 'setRefid(',
 'append(',
 'append(',
 'append(',
 'size(',
 'append(',
 'append(',
 'getLastModifiedTime(',
 'append(',
 'stream(',
 'sorted(',
 'comparing(',
 'collect(',
 'toList(',
 'toLongString(',
 'getFile(',
 'toPath(',
 'exists(',
 'append(',
 'append(',
 'isDirectory(',
 'walk(',
 'sorted(',
 'filter(',
 'isDirectory(',
 'forEach(',
 'toString(',
 'exists(',
 'toPath(',
 'readString(',
 'equals(',
 'isEmpty(',
 'isEmpty(',
 'perform(',
 'toPath(',
 'createDirectories(',
 'getParent(',
 'writeString(',
 'getApplication(',
 'getService(',
 'copyBean(',
 'areEqual(',

In [7]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['module'])):
        text = f"{example['prompt'][i]}{example['module'][i]}"
        output_texts.append(text)
    return output_texts

In [8]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM,
)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./models/t5_fine_tune/",
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.24,
)


trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    args=training_args,
    max_seq_length=512,
    dataset_batch_size=1

)

Map:   0%|          | 0/814796 [00:00<?, ? examples/s]

In [9]:
torch.cuda.empty_cache()
gc.collect()

76

In [ ]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.035500
1000,0.005800
1500,0.005900
2000,0.005100
2500,0.003500
3000,0.004200
3500,0.004300
4000,0.004100
4500,0.004700
5000,0.002100


Step,Training Loss
500,0.035500
1000,0.005800
1500,0.005900
2000,0.005100
2500,0.003500
3000,0.004200
3500,0.004300
4000,0.004100
4500,0.004700
5000,0.002100


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model("./models/t5_fine_tune//v2")

In [10]:
from transformers import AutoConfig
from peft import PeftConfig, PeftModel

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/codet5-small")

In [21]:
model = model.from_pretrained( "./model2")

A model fully trained in 6 epochs is used here, although the jupyter has not been preserved since that time.

In [22]:
model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=512, out_features=512, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=512, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=512, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): Linear(in_features=512, out_features=512, bias=False)
              

In [84]:
def generate(prompt):
  data = tokenizer( prompt, return_tensors="pt")
  data = {k: v.to(model.device) for k, v in data.items()}
  output_ids = model.generate(
      **data,  do_sample=True ,temperature=0.2, max_new_tokens=30,  repetition_penalty=1.03, no_repeat_ngram_size=2
  )[0]

  out = tokenizer.decode(output_ids.tolist(),skip_special_tokens=True)
  out = out.replace("<s>","").replace("</s>","")
  return out

In [28]:
def generate_text(prompt,  max_length=150):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    generated_ids = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return generated_text

# Загрузка модели и токенизатора


# Пример использования
prompt = "IconManager."

generated_text = generate_text(prompt)
print(generated_text)

IconManager.getIcon ( ) ; IconManager. getIconClass(


In [74]:
promt =  "IconManager."

In [30]:
print(generate(promt)) #Absolutely right

IconManager.getInstance(


In [83]:
print(generate(promt))

IconManager.getIcon ( ) ;


In [32]:
print(generate(promt)) #Absolutely right

IconManager.getInstance(


In [58]:
prompt = " { System.setProperty(""temp.dir"", temporaryFolder."

In [63]:
print(generate(prompt)) #Almost right

{ System.setProperty(temp.dir, temporaryFolder.getCanonicalPath( ) .


In [65]:
print(generate(prompt))

 { System.setProperty(temp.dir, temporaryFolder. .


As we can see on large prompts, sometimes the model refuses to output something, but it needs to be caught

In [20]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.8 MB/s eta 0:00:00


In [44]:
import Levenshtein
lev = 0
i = 0

for prompt in dataset["prompt"]:
  reference = dataset['module'][i]
  an = " ".join(prompt.split()[5:])
  candidate = generate(an)
  distance = Levenshtein.distance(reference, candidate)
  i+=1
  lev += distance/max(len(reference), len(candidate))

In [67]:
print(lev/i)
print(282019/814796)

0.34612221954943323


The estimate is not bad, although it shows that in about a third of cases the model refuses to generate anything. When calculating without fantuning, I got 0.93.

In [86]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
score = 0
if (0):
  for prompt in dataset["prompt"]:
    reference = dataset['module'][i]
    an = " ".join(prompt.split()[5:])
    candidate = generate(an)
    score += sentence_bleu(reference, candidate)


In [88]:
print(score/i)

0.2233921128724245


In my opinion, for an under-trained model, a very good score indicates that in about five cases the model gives an absolutely correct answer. Before the funtuning, the score is 0.